# Введение в регрессионный анализ
## Семинар 6. Модели с дамми-переменными (фиктивными переменными)
*Алла Тамбовцева*

Импортируем необходимые для работы библиотеки и функции:

In [2]:
import pandas as pd
from statsmodels.formula.api import ols

Запустим строку кода, которая будет скрывать сообщения с предупреждениями в случае, если они возникают при создании новых столбцов на основе старых:

In [3]:
pd.set_option('mode.chained_assignment', None)

## Сюжет 1: парная регрессия с одной дамми-переменной

Продолжим работать с данными из файла `english.csv` с прошлого практикума. 

**Напоминание про исследование.** Участникам эксперимента предлагают определить, является ли слово, которое они видят на экране, реально существующим в языке или нет (настоящее это слово или что-то выдуманное по правилам языка). Если участники узнают слово, они должны прочитать его вслух и нажать на специальную кнопку. Время, затраченное на узнавание слова фиксируется и измеряется в милисекундах: насколько быстро человек нажал на кнопку (реальное слово или нет, *lexical decision*), или прочитал слово (*word naming*).

В файле отобраны только случаи с реально существующими словами. Переменные в файле:

* `AgeSubject`: молодой участник эксперимента или нет;
* `WordCategory`: часть речи (`N` – существительное, `V` – глагол);
* `RTlexdec`: время в милисекундах, затраченное на узнавание слова;
* `RTnaming`: время в милисекундах, затраченное на называние слова;
* `WrittenFrequency`: мера того, насколько часто слово встречается в письменных текстах;
* `LengthInLetters`: длина слова в буквах;
* `FamilySize`: мера того, насколько богата морфологическая семья слова (как много однокоренных слов с разной частью речи);
* `NumberSimplexSynsets`: мера того, насколько у слова много синонимов.

Загрузим данные и выберем только молодых участников эксперимента:

In [4]:
eng = pd.read_csv("english.csv")
df = eng[eng["AgeSubject"] == "young"]

Поскольку с данными мы уже познакомились в прошлый раз, перейдём сразу к моделям. Предположим, мы хотим построить модель, которая предсказывает время, затрачиваемое на узнавание слова, на основе его части речи. Давайте сначала пойдём по самому логичному пути и создадим бинарную переменную для части речи, где 1 соответствует глаголу, а 0 – существительному.

### Задача 1

Создайте бинарную переменную, где 1 соответствует глаголу, а 0 – существительному, и сохраните её в столбец `Verb` датафрейма `df`.

In [5]:
# df["WordCategory"] == "V" дает столбец из True/False
# превращаем его в integer (True -> 1, False -> 0)

df["Verb"] = (df["WordCategory"] == "V").astype(int)

### Задача 2

Используя созданную бинарную переменную, постройте парную линейную модель, которая предсказывает время, затрачиваемое на узнавание слова, на основе его части речи. Проинтерпретируйте полученные результаты.

In [6]:
model01 = ols("RTlexdec ~ Verb", data = df).fit()
print(model01.summary())

                            OLS Regression Results                            
Dep. Variable:               RTlexdec   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     10.03
Date:                Mon, 14 Oct 2024   Prob (F-statistic):            0.00156
Time:                        02:15:33   Log-Likelihood:                 1883.5
No. Observations:                2284   AIC:                            -3763.
Df Residuals:                    2282   BIC:                            -3752.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.4446      0.003   2314.049      0.0

**Интерпретация.** Качество модели, если смотреть только на $R^2$, кажется низким, однако для моделей, где независимая переменная бинарная, это нормально. $R^2$ зависит, в частности, от дисперсии независимой переменной, а раз у нее всего два значения, 0 и 1, очень высокой она не будет (разнообразие значений, если их всего два, не может быть большим).

* Уравнение модели: $\widehat{\text{RTlexdec}} = 6.44 - 0.01 \times \text{Verb}_i$. 
* Если $\text{Verb}_i$ равен 0, то среднее время узнавания слова, согласно модели, равно 6.44 милисекундам. Содержательно это означает, что среднее время узнавания существительного (раз `Verb` равен 0), согласно модели, равно 6.44 мс. 
* Так как `Verb` – качественный показатель (глагол или существительное), об увеличении его на единицу говорить некорректно. Коэффициент при $\text{Verb}_i$ показывает разницу в средних значениях зависимой переменной в двух группах. Среднее время узнавания глагола на 0.01 мс ниже, чем среднее время узнавания существительного, глаголы угадываются немного быстрее.
* Несмотря на то, что значение оценки коэффициента при $\text{Verb}_i$ кажется маленьким, он значимо отличается от нуля. Если посмотрим на наблюдаемое значение t-статистики (`t = -3.167`) и p-value (`P>|t| = 0.002`), мы сможем заключить, что гипотеза $H_0: b_1 = 0$ отвергается на любом уровне доверия (90%, 95%, 99%). 
* Для уровня доверия 95% можем дополнительно посмотреть на доверительный интервал для оценки коэффициента (`-0.024  -0.006`) и отметить, что он не накрывает 0, что тоже свидетельствует о несостоятельности гипотезы о равенстве коэффициента нулю.

На самом деле, функция `ols()` умеет работать и с исходными качественными переменными. Если в качестве независимой переменной в уравнении внутри `ols()` используется текстовый столбец, функция сама создаст в рамках модели необходимую бинарную дамми-переменную. Проверим!

### Задача 3

Используя исходную переменную `WordCategory`, постройте парную линейную модель, которая предсказывает время, затрачиваемое на узнавание слова, на основе его части речи. Проинтерпретируйте полученные результаты.

In [7]:
model02 = ols("RTlexdec ~ WordCategory", data = df).fit()
print(model02.summary())

                            OLS Regression Results                            
Dep. Variable:               RTlexdec   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     10.03
Date:                Mon, 14 Oct 2024   Prob (F-statistic):            0.00156
Time:                        02:39:47   Log-Likelihood:                 1883.5
No. Observations:                2284   AIC:                            -3763.
Df Residuals:                    2282   BIC:                            -3752.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             6.4446      0.00

**Интерпретация.** Та же самая, что и выше. Тут важно понимать, что запись `WordCategory[T.V]` в выдаче означает следующее: столбец на основе `WordCategory`, где `True` (сокращено до `T.`) соответствует значению `V`, а значение `False` – всем остальным. Другими словами, мы получили такую же модель, что и выше, но для ее получения нам не пришлось предварительно создавать бинарный столбец, где 1 (оно же `True`) соответствует глаголам. Python самостоятельно проделал эту операцию в рамках запуска `ols()`.

Мы получили уравнение $\widehat{\text{RTlexdec}} = 6.44 - 0.01 \times \text{WordCategory[T.V]}_i$, которое легко сводится к $\widehat{\text{RTlexdec}} = 6.44 - 0.01 \times \text{Verb}_i$. 

По умолчанию Python выбирает в качестве базовой категории ту, которая идёт первой по алфавиту. Давайте в качестве базовой категории выберем глагол (`V`). Как это сделать? Учесть в рамках функции `ols()` это не получится, нужно перекодировать данные.

### Задача 4

Используя функцию `Categorical()` из `pandas`, измените столбец `WordCategory` таким образом, чтобы значение `V` считалось первым, а значение `N` – вторым.

Логика запуска функции:

* на первом месте указываем название столбца, который перекодируем;
* на втором месте – список значений в том порядке, который нас устраивает;
* на третьем месте – аргумент `ordered=True`, чтобы зафиксировать порядок следования.

In [8]:
# хотим, чтобы Python понимал, что V – это первое,
# а N – второе

df["WordCategory"] = pd.Categorical(df["WordCategory"], ["V", "N"], ordered=True)

In [9]:
# визуально выглядит так же, как и ранее, 
# но теперь для Python столбец WordCategory упорядоченный

df.head()

,AgeSubject,WordCategory,RTlexdec,RTnaming,WrittenFrequency,LengthInLetters,FamilySize,NumberSimplexSynsets,Verb
0,young,N,6.543754,6.145044,3.912023,3,1.386294,0.693147,0
1,young,N,6.397596,6.246882,4.521789,5,1.386294,1.098612,0
2,young,N,6.304942,6.143756,6.505784,6,1.609438,2.484907,0
3,young,N,6.424221,6.131878,5.017280,4,1.945910,1.098612,0
4,young,N,6.450597,6.198479,4.890349,4,2.197225,2.484907,0


### Задача 5

Используя обновлённую переменную `WordCategory`, постройте парную линейную модель, которая предсказывает время, затрачиваемое на узнавание слова, на основе его части речи. Сравните результаты с выдачей модели из задачи 3 и прокомментируйте различия.

In [10]:
model03 = ols("RTlexdec ~ WordCategory", data = df).fit()
print(model03.summary())

                            OLS Regression Results                            
Dep. Variable:               RTlexdec   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     10.03
Date:                Mon, 14 Oct 2024   Prob (F-statistic):            0.00156
Time:                        02:46:18   Log-Likelihood:                 1883.5
No. Observations:                2284   AIC:                            -3763.
Df Residuals:                    2282   BIC:                            -3752.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             6.4299      0.00

**Интерпретация.** Мы поменяли категории местами, поэтому:

* знак при `WordCategory` изменился по сравнению с предыдущей моделью;
* оценка константы $\hat{b}_0$ тоже изменилась;
* $R^2$ и остальные характеристики модели остались прежними.

Итого:

* Уравнение модели: $\widehat{\text{RTlexdec}} = 6.43 + 0.01 \times \text{WordCategory[T.N]}_i$, которое легко сводится к $\widehat{\text{RTlexdec}} = 6.43 + 0.01 \times \text{Noun}_i$. 
* Согласно модели, среднее время узнавания слова, если оно является глаголом, равно 6.43 милисекундам. Среднее время узнавания существительного на 0.01 мс выше, чем среднее время угадывания глагола, глаголы угадываются немного быстрее.
* Так как мы уже выяснили, что оценка коэффициента при `WordCategory`, равная 0.01, отвечает за разницу в среднем времени узнавания существительных и глаголов, несложно понять, почему значения `Intercept` в модели из задачи 3 и в модели из этой задачи равны соответственно 6.44 и 6.43.

## Сюжет 2: множественная модель с одной дамми-переменной

В прошлый раз мы строили множественную регрессионную модель, которая предсказывала время, затрачиваемое на узнавание слова, в зависимости от его грамматических характеристик, выраженных в числовом виде. Построим модель, которая включает не только количественные, но и качественные характеристики слова!

Постройте линейную модель, которая объясняет, как время, затрачиваемое на узнавание слова, зависит от его встречаемости в письменных текстах, размера морфологической семьи и части речи. Проинтерпретируйте полученные результаты.

In [11]:
model04 = ols("RTlexdec ~ WordCategory + WrittenFrequency + FamilySize", data = df).fit()
print(model04.summary())

                            OLS Regression Results                            
Dep. Variable:               RTlexdec   R-squared:                       0.431
Model:                            OLS   Adj. R-squared:                  0.431
Method:                 Least Squares   F-statistic:                     576.4
Date:                Mon, 14 Oct 2024   Prob (F-statistic):          8.85e-279
Time:                        03:06:03   Log-Likelihood:                 2523.1
No. Observations:                2284   AIC:                            -5038.
Df Residuals:                    2280   BIC:                            -5015.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             6.6240      0.00

**Интерпретация.**

* Уравнение модели: $$\widehat{\text{RTlexdec}} = 6.62 + 0.01 \times \text{WordCategory[T.N]}_i - 0.03 \times \text{WrittenFrequency}_i - 0.02 \times \text{FamilySize}_i.$$
* Согласно модели, среднее время узнавания глагола (`WordCategory[T.N]` равно 0), который выдуман и не встречается в письменных текстах (`WrittenFrequency` равно 0) и у которого нет однокоренных слов (`FamilySize` равно 0) составляет 6.62 милисекунды. Ситуация в данном случае не очень реалистичная, но это формальный смысл константы `Intercept`.
* При прочих равных условиях, среднее время узнавания существительных на 0.01 мс выше среднего времени узнавания глаголов. То есть, если мы будем сравнивать среднее время узнавания двух слов, которые встречаются в письменных текстах с одинаковой частотой и которые имеют одинаковое число однокоренных слов, значение для существительного будет выше на 0.01 мс.
* При прочих равных условиях, при увеличении встречаемости слова в письменных текстах на единицу, время узнавания слова, в среднем, уменьшается на 0.03 мс. При прочих равных условиях здесь – если мы сравниваем слова одной и той же части речи и с одинаковым количеством однокоренных слов.
* При прочих равных условиях, при увеличении числа однокоренных слов на единицу, время узнавания слова, в среднем, уменьшается на 0.02 мс. При прочих равных условиях здесь – если мы сравниваем слова одной и той же части речи, одинаково часто встречающиеся в письменных текстах.
* Так как наблюдаемые значения t-статистики для всех оценок коэффициентов по модулю превышают 3, и p-value везде равно 0, на любом уровне доверия (90%, 95%, 99%) или любом уровне значимости (10%, 5%, 1%) эффект каждой независимой переменной можно считать значимо отличным от нуля.
* Качество модели нельзя назвать плохим, модель объясняет 43% дисперсии зависимой переменной. Другими словами, изменчивость времени угадывания слова на 43% можно объяснить изменчивостью встречаемости слова в письменных текстах и количества однокоренных слов, а также принадлежностью к разным частям речи.

## Сюжет 3: множественная модель с несколькими дамми-переменными

Давайте частично воспроизведем модель из статьи *E.Patacchini et al "Information Transmission in a Social Network: A Field Experiment"* и выясним, зависит ли успешность групповой работы от типа устройства сети сообщества, при условии, что тип сети определяет характер передачи информации, которая приближает группу к выигрышу.

В этой статье описывается экспериментальная игра, в которой участвовали ученики одной школы Италии. Школьников случайным образом распределяли на группы по 5 человек и давали каждому индивидуальное задание – угадать все цвета элементов одежды персонажа (шапка, рубашка, брюки, перчатки, ботинки), который показывался им на картинке в приложении. Каждому участнику был достоверно известен только цвет какого-то одного элемента одежды. Идея понятна: в идеальной ситуации в группе из пяти человек каждый может поделиться своей частью информации с другим, узнать все пять элементов одежды и остаться в выигрыше. Но все не так просто. Школьники не знают, с кем они находятся в группе, эта информация им доступна лишь частично: кто-то знает имена трех членов группы, кто-то – только двух, кто-то – только одного. Никакого чата в игре нет и участники не могут его создать, они могут лишь вживую, зная имя человека, обменяться с ним информацией.

Таким образом, группы из пяти человек устроены по-разному, по разным типам сети.

* Первый тип сети: довольно базовый, участники знают одного-двух участников группы, при этом есть наиболее осведомленный член группы, который знает трех участников.

* Второй тип сети: явное преимущество у одного члена группы, он знает имена всех четырех участников, остальные – лишь двоих.

* Третий тип сети: более равномерное число связей, все участники, кроме одного, знают имена двух-трех участников.

За угадывания цветов игроки получают баллы, попытки с неправильными ответами штрафуются. Участники, набравшие наибольшее число очков, могли выиграть суммы от 0 до 15 евро, выигрыш вручался в виде подарочных карт от Amazon на определенную сумму.

**Итого:** интересно посмотреть, каким образом тип сети (а, следовательно, и конфигурация системы передачи информации) сказывается на групповом выигрыше. Если смотреть только на устройство сети, кажется, что вторая сеть, где есть участник, знающий всех, идеальна, потому что он будет центральным «узлом» передачи информации для всех игроков. Но в реальности, где рациональный выбор все же существует, все может оказаться иначе: центральному игроку, знающему всех, может быть невыгодно служить источником информации для остальных, так как передавая правдивую информацию он, в том числе, повышает шансы других угадать больше ответов и получить выигрыш.

Загрузим данные с результатами эксперимента:

In [12]:
data = pd.read_csv("networks.csv")
data.head()

,Unnamed: 0,phase,character,idusers,idnetwork,hint_cloth,hint_col,hat_guess,shirt_guess,pant_guess,...,score_glove,score,num_corrects,num_mistakes,grey,winner,grp_act_num,grp_num_corrects,grp_num_mistakes,pos
0,1,1.0,Freddie,358,1,cappello,nero,9.0,5.0,4.0,...,10.0,50.0,5.0,0.0,0.0,1.0,5.0,25.0,0.0,a
1,3,1.0,Piergiorgio,603,1,guanti,bianco,10.0,10.0,2.0,...,10.0,20.0,2.0,0.0,3.0,1.0,5.0,11.0,2.0,a
2,8,1.0,Marshall,447,1,cappello,bianco,8.0,2.0,2.0,...,10.0,50.0,5.0,0.0,0.0,1.0,5.0,21.0,1.0,a
3,14,1.0,Alberto,184,1,guanti,blu,4.0,5.0,7.0,...,10.0,5.0,2.0,3.0,0.0,0.0,5.0,16.0,8.0,a
4,18,1.0,Teseo,1417,1,cappello,arancione,2.0,3.0,1.0,...,10.0,35.0,4.0,1.0,0.0,1.0,5.0,15.0,4.0,a


Итак, построим линейную модель, где зависимая переменная – число правильных ответов на группу, а независимая – тип сети:

In [13]:
model_nw = ols("grp_num_corrects ~ idnetwork", data).fit()
print(model_nw.summary())

                            OLS Regression Results                            
Dep. Variable:       grp_num_corrects   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     17.05
Date:                Mon, 14 Oct 2024   Prob (F-statistic):           4.12e-05
Time:                        03:44:17   Log-Likelihood:                -2064.8
No. Observations:                 660   AIC:                             4134.
Df Residuals:                     658   BIC:                             4143.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.7510      0.589     25.036      0.0

Модель, которую мы только что построили, некорректная. Почему? Посмотрим на `idnetwork`. По-хорошему, у нас в модели должно быть две фиктивные переменные, одна для сравнения второго типа с первым, вторая – для сравнения третьего типа с первым. Здесь такого не происходит, потому что тип столбца `idnetwork` целочисленный, и Python воспринимает его как полноценную числовую переменную. Как это поправить? Можно изменить тип переменной в датафрейме глобально, а можно просто учесть это в формуле `ols()`:

In [14]:
# C() – от Categorical, то есть
# смотри на переменную как на качественную, категориальную

mod1 = ols("grp_num_corrects ~ C(idnetwork)", data).fit()
print(mod1.summary())

                            OLS Regression Results                            
Dep. Variable:       grp_num_corrects   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     15.89
Date:                Mon, 14 Oct 2024   Prob (F-statistic):           1.82e-07
Time:                        03:44:20   Log-Likelihood:                -2057.7
No. Observations:                 660   AIC:                             4121.
Df Residuals:                     657   BIC:                             4135.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            16.4615      0.39

Теперь все нормально. Проинтерпретируем полученную модель!

* Уравнение модели: $$\widehat{\text{grp_num_corrects}_i} = 16.46 - 0.75 \times \text{C(idnetwork)[T.2]}_i + 2 \times \text{C(idnetwork)[T.3]}_i,$$ перепишем для простоты:
$$
\widehat{\text{grp_num_corrects}_i} = 16.46 - 0.75 \times \text{Network2}_i + 2 \times \text{Network3}_i
$$

* `Intercept`: константа модели, содержательно это среднее значение зависимой переменной в базовой категории. Базовая категория здесь – это группы, устроенные по первому типу сети, с `idnetwork` равной 1, так как эта категория «выкинута» из выдачи. То есть, мы можем сказать, что среднее общегрупповое число верных ответов в группах, устроенных по типу первой сети, примерно равно 16.

* `C(idnetwork)[T.2]`: коэффициент при втором типе сети, показывает разницу средних значений зависимой переменной в этой категории и базовой категории. Можем сказать, что среднее общегрупповое число верных ответов в группах, устроенных по типу второй сети, ниже на 0.75 (то есть примерно на 1), чем среднее число верных ответов в группах, устроенных по типу первой сети. Однако этот коэффициент является незначимым на любом разумном уровне значимости (и на 5%, и на 10%) в силу высокого p-value 0.169, поэтому интерпретировать его особого смысла нет.

* `C(idnetwork)[T.3]`: коэффициент при третьем типе сети, показывает разницу средних значений зависимой переменной в этой категории и базовой категории. Можем сказать, что среднее общегрупповое число верных ответов в группах, устроенных по типу третьей сети, на 2 выше, чем среднее число верных ответов в группах, устроенных по типу первой сети. Оценка коэффициента в данном случае статистически значима на 5%-ном уровне значимости (и на 1%-ном тоже), поэтому различия точно есть. Выходит, третий тип сети самый выгодный, если нас интересует максимизация выигрыша всей группы.